In [1]:
from config import *
from models import *

#language_model = LanguageModel()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/melkor/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
pybullet build time: Oct 24 2022 03:35:20


In [ ]:
import torch
import torch.nn as nn

class BoxTransition(nn.Module):
    def __init__():
        super().__init__()
    def forward(self, x): return x